In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [96]:
penguins = sns.load_dataset("penguins")

In [97]:
penguins.head()

species     island  bill_length_mm  bill_depth_mm  flipper_length_mm  \
0  Adelie  Torgersen            39.1           18.7              181.0   
1  Adelie  Torgersen            39.5           17.4              186.0   
2  Adelie  Torgersen            40.3           18.0              195.0   
3  Adelie  Torgersen             NaN            NaN                NaN   
4  Adelie  Torgersen            36.7           19.3              193.0   

   body_mass_g     sex  
0       3750.0    Male  
1       3800.0  Female  
2       3250.0  Female  
3          NaN     NaN  
4       3450.0  Female

In [98]:
penguins.shape

(344, 7)

### Remove missing values

In [99]:
penguins.dropna(inplace=True)

In [100]:
penguins.shape

(333, 7)

### Feature selection

In [102]:
columns = list(penguins.columns)
# columns.remove('island')
# columns.remove('sex')

penguins = penguins[columns]

### Split Train Test

In [103]:
from sklearn.model_selection import train_test_split

In [104]:
train, test = train_test_split(penguins, test_size = 133)
test = test.copy()

In [105]:
train.shape

(200, 7)

In [106]:
test.shape

(133, 7)

### Train features vs target

In [107]:
train_features = train.iloc[:,1:]
train_target = train['species']

### Scaling

In [108]:
from sklearn.preprocessing import StandardScaler

In [109]:
train_numeric_features = train_features.select_dtypes(include='number')
train_numeric_features

bill_length_mm  bill_depth_mm  flipper_length_mm  body_mass_g
1              39.5           17.4              186.0       3800.0
12             41.1           17.6              182.0       3200.0
250            47.3           15.3              222.0       5250.0
44             37.0           16.9              185.0       3000.0
229            46.8           15.4              215.0       5150.0
..              ...            ...                ...          ...
204            45.7           17.3              193.0       3600.0
116            38.6           17.0              188.0       2900.0
135            41.1           17.5              190.0       3900.0
160            46.0           18.9              195.0       4150.0
21             37.7           18.7              180.0       3600.0

[200 rows x 4 columns]

In [110]:
scaler = StandardScaler()

scaler.fit(train_numeric_features)

X_train_scaled = scaler.transform(train_numeric_features)

In [111]:
X_train_scaled

array([[-0.82480214,  0.10742823, -1.09433127, -0.52712705],
       [-0.52874305,  0.20644042, -1.38015074, -1.2724024 ],
       [ 0.61848596, -0.93219981,  1.4780439 ,  1.27395506],
       [-1.28739449, -0.14010225, -1.16578614, -1.52082752],
       [ 0.52596749, -0.88269371,  0.97785984,  1.1497425 ],
       [-0.91732061, -0.04109006, -0.73705694, -1.45872124],
       [ 0.95155245, -1.32824858,  0.76349524,  0.49762656],
       [ 1.48815956,  1.74112943,  0.62058551,  0.71499854],
       [-0.12166179, -1.57577907,  0.47767578,  0.43552028],
       [ 1.11808569,  1.14705627, -0.37978261, -0.40291449],
       [ 0.54447119, -0.28862055, -0.66560208, -1.8934652 ],
       [ 0.22990839, -0.68466932,  0.97785984,  1.33606134],
       [ 1.28461893,  0.35495872, -0.37978261, -0.68239275],
       [-0.86180953,  1.69162334, -0.80851181, -0.71344589],
       [ 0.97005614, -0.9817059 ,  1.40658904,  2.57818693],
       [-0.45472827,  0.5529831 , -0.45123748,  0.09393575],
       [-0.54724674,  1.

### One-hot coding

In [112]:
from sklearn.preprocessing import OneHotEncoder

In [113]:
train_categorical_features = train_features.select_dtypes(include=object)
train_categorical_features

island     sex
1    Torgersen  Female
12   Torgersen  Female
250     Biscoe    Male
44       Dream  Female
229     Biscoe    Male
..         ...     ...
204      Dream  Female
116  Torgersen  Female
135      Dream    Male
160      Dream  Female
21      Biscoe    Male

[200 rows x 2 columns]

In [114]:
onehot_encoders = {}
for column in train_categorical_features:
    onehot_encoder = OneHotEncoder()
    onehot_encoder.fit(train_categorical_features[column].values.reshape(-1,1))

    onehot_encoders[column] = onehot_encoder

In [115]:
Xs = []
for column, onehot_encoder in onehot_encoders.items():
    
    X = onehot_encoder.transform(train_categorical_features[column].values.reshape(-1,1)).toarray()
    Xs.append(X)
    
X_train_encoded = np.concatenate(Xs, axis=1)

In [116]:
X_train = np.concatenate((X_train_scaled, X_train_encoded), axis=1)
X_train

array([[-0.82480214,  0.10742823, -1.09433127, ...,  1.        ,
         1.        ,  0.        ],
       [-0.52874305,  0.20644042, -1.38015074, ...,  1.        ,
         1.        ,  0.        ],
       [ 0.61848596, -0.93219981,  1.4780439 , ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [-0.52874305,  0.15693433, -0.80851181, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.37793794,  0.85001969, -0.45123748, ...,  0.        ,
         1.        ,  0.        ],
       [-1.15786863,  0.75100749, -1.52306047, ...,  0.        ,
         0.        ,  1.        ]])

In [117]:
X_train.shape

(200, 9)

### Test set

In [118]:
test_features = test.iloc[:,1:]
test_target = test['species']

In [119]:
test_numeric_features = test_features.select_dtypes(include='number')
test_numeric_features

bill_length_mm  bill_depth_mm  flipper_length_mm  body_mass_g
167            50.5           19.6              201.0       4050.0
296            47.5           14.2              209.0       4600.0
86             36.3           19.5              190.0       3800.0
98             33.1           16.1              178.0       2900.0
150            36.0           17.1              187.0       3700.0
..              ...            ...                ...          ...
289            50.7           15.0              223.0       5550.0
326            41.7           14.7              210.0       4700.0
43             44.1           19.7              196.0       4400.0
218            50.8           19.0              210.0       4100.0
73             45.8           18.9              197.0       4150.0

[133 rows x 4 columns]

In [120]:
X_test_scaled = scaler.transform(test_numeric_features)

In [121]:
test_categorical_features = test_features.select_dtypes(include=object)
test_categorical_features

island     sex
167      Dream    Male
296     Biscoe  Female
86       Dream    Male
98       Dream  Female
150      Dream  Female
..         ...     ...
289     Biscoe    Male
326     Biscoe  Female
43       Dream    Male
218      Dream    Male
73   Torgersen    Male

[133 rows x 2 columns]

In [122]:
Xs = []
for column, onehot_encoder in onehot_encoders.items():
    
    X = onehot_encoder.transform(test_categorical_features[column].values.reshape(-1,1)).toarray()
    Xs.append(X)
    
X_test_encoded = np.concatenate(Xs, axis=1)

In [123]:
X_test = np.concatenate((X_test_scaled, X_test_encoded), axis=1)
X_test

array([[ 1.21060416,  1.19656237, -0.02250828, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.65549335, -1.47676687,  0.54913065, ...,  0.        ,
         1.        ,  0.        ],
       [-1.41692034,  1.14705627, -0.80851181, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 0.02636776,  1.24606846, -0.37978261, ...,  0.        ,
         0.        ,  1.        ],
       [ 1.26611524,  0.89952578,  0.62058551, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.34093056,  0.85001969, -0.30832775, ...,  1.        ,
         0.        ,  1.        ]])

In [124]:
X_test.shape

(133, 9)

### k Nearest Neighbor

In [125]:
from sklearn.neighbors import KNeighborsClassifier

In [126]:
knn_classifier = KNeighborsClassifier(n_neighbors = 3)
knn_classifier.fit(X_train, train_target)

KNeighborsClassifier(n_neighbors=3)

In [127]:
y_predict = knn_classifier.predict(X_test)

In [128]:
test['knn_predicted'] = y_predict
test['knn_correct'] = test['knn_predicted'] == test['species']

In [129]:
test

species     island  bill_length_mm  bill_depth_mm  flipper_length_mm  \
167  Chinstrap      Dream            50.5           19.6              201.0   
296     Gentoo     Biscoe            47.5           14.2              209.0   
86      Adelie      Dream            36.3           19.5              190.0   
98      Adelie      Dream            33.1           16.1              178.0   
150     Adelie      Dream            36.0           17.1              187.0   
..         ...        ...             ...            ...                ...   
289     Gentoo     Biscoe            50.7           15.0              223.0   
326     Gentoo     Biscoe            41.7           14.7              210.0   
43      Adelie      Dream            44.1           19.7              196.0   
218  Chinstrap      Dream            50.8           19.0              210.0   
73      Adelie  Torgersen            45.8           18.9              197.0   

     body_mass_g     sex knn_predicted  knn_correct  
167       4050.0    Male     Chinstrap         True  
296       4600.0  Female        Gentoo         True  
86        3800.0    Male        Adelie         True  
98        2900.0  Female        Adelie         True  
150       3700.0  Female        Adelie         True  
..           ...     ...           ...          ...  
289       5550.0    Male        Gentoo         True  
326       4700.0  Female        Gentoo         True  
43        4400.0    Male        Adelie         True  
218       4100.0    Male     Chinstrap         True  
73        4150.0    Male        Adelie         True  

[133 rows x 9 columns]

In [130]:
test['knn_correct'].value_counts()

True     131
False      2
Name: knn_correct, dtype: int64

### Decision Tree

In [131]:
from sklearn.tree import DecisionTreeClassifier

In [132]:
dt_classifier = DecisionTreeClassifier()
dt_classifier.fit(X_train, train_target)

DecisionTreeClassifier()

In [133]:
y_predict = dt_classifier.predict(X_test)

In [134]:
test['dt_predicted'] = y_predict
test['dt_correct'] = test['dt_predicted'] == test['species']

In [135]:
test

species     island  bill_length_mm  bill_depth_mm  flipper_length_mm  \
167  Chinstrap      Dream            50.5           19.6              201.0   
296     Gentoo     Biscoe            47.5           14.2              209.0   
86      Adelie      Dream            36.3           19.5              190.0   
98      Adelie      Dream            33.1           16.1              178.0   
150     Adelie      Dream            36.0           17.1              187.0   
..         ...        ...             ...            ...                ...   
289     Gentoo     Biscoe            50.7           15.0              223.0   
326     Gentoo     Biscoe            41.7           14.7              210.0   
43      Adelie      Dream            44.1           19.7              196.0   
218  Chinstrap      Dream            50.8           19.0              210.0   
73      Adelie  Torgersen            45.8           18.9              197.0   

     body_mass_g     sex knn_predicted  knn_correct dt_predicted  dt_correct  
167       4050.0    Male     Chinstrap         True    Chinstrap        True  
296       4600.0  Female        Gentoo         True       Gentoo        True  
86        3800.0    Male        Adelie         True       Adelie        True  
98        2900.0  Female        Adelie         True       Adelie        True  
150       3700.0  Female        Adelie         True       Adelie        True  
..           ...     ...           ...          ...          ...         ...  
289       5550.0    Male        Gentoo         True       Gentoo        True  
326       4700.0  Female        Gentoo         True       Gentoo        True  
43        4400.0    Male        Adelie         True    Chinstrap       False  
218       4100.0    Male     Chinstrap         True    Chinstrap        True  
73        4150.0    Male        Adelie         True       Adelie        True  

[133 rows x 11 columns]

In [136]:
test['dt_correct'].value_counts()

True     129
False      4
Name: dt_correct, dtype: int64